In [31]:
# import cassandra python package
import cassandra

In [32]:
# set up DB connection
from cassandra.cluster import Cluster
try:
    cluster =  Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e)

In [33]:
# Test if connection works, a No keyspace error is expected
try:
    session.execute("""select * from music_library""")
except Exception as e:
    print(e)

Error from server: code=2200 [Invalid query] message="No keyspace has been specified. USE a keyspace, or explicitly specify keyspace.tablename"


In [34]:
# Create a keyspace - same as schema or database in RDBMS
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS udacity WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")
except Exception as e:
    print(e)

In [35]:
# use the created keyspace
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

In [36]:
# create a table
query = "CREATE TABLE IF NOT EXISTS music_library"
query+= "(year int, artist_name text, album_name text, PRIMARY KEY (year, artist_name))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [37]:
# insert records into table
query = "INSERT INTO music_library (year, artist_name, album_name)"
query = query + "VALUES(%s, %s, %s)"
try: 
    session.execute(query, (1970, "The Beatles", "Let it Be"))
except Exception as e:
    print(e)
try: 
    session.execute(query, (1965, "The Beatles", "Rubber Soul"))
except Exception as e:
    print(e)

In [38]:
# verify that records where inserted
query = "select count(*) from music_library"
try:
    count = session.execute(query)
except Exception as e:
    print(e)
print(count.one())

Row(count=2)


In [39]:
# perform selection on table
query = "select * from music_library WHERE YEAR=1970"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.year, row.album_name, row.artist_name)

1970 Let it Be The Beatles


In [40]:
# Drop the table just for housekeeping - Use this with discretion on a live table
query = "drop table music_library"
try:
    rows= session.execute(query)
except Exception as e:
    print(e)

In [42]:
# close session and cluster connection
session.shutdown()
cluster.shutdown()